## Convert smooth Roemmich-Gilson Argo Climatology data from mat to zarr
---
**Author**: Hillary A. Scannell (scanh@uw.edu)

**Last updates**: 25 November 2020

This data was processed on July 17, 2020 by Greg Johnson (NOAA/PMEL). It contains the smooth model coefficients to reconstruct the total smooth fields for subsurface temperature and salinity on isobars. 

> **Dataset**: Roemmich, D. and J. Gilson, 2009: The 2004-2008 mean and annual cycle of temperature, salinity, and steric height in the global ocean from the Argo Program. Progress in Oceanography, 82, 81-100. 

In [1]:
import numpy as np
import h5py
import xarray as xr

In [ ]:
with h5py.File('/glade/scratch/scanh/grl_2020_data/te_sa_model_no_nino_smoothed_copy.mat', 'r') as f:
    print(list(f.keys()))
    dyr = np.squeeze(np.array(f['dyr']))
    bathymetry_mask = np.array(f['ba'])
    lat = np.squeeze(np.array(f['lat']))
    lon = np.squeeze(np.array(f['lon']))
    pressure = np.squeeze(np.array(f['pr']))
    argo_salinity_anomaly_filt = np.array(f['sa_resff'])
    argo_temperature_anomaly_filt = np.array(f['te_resff'])
    argo_te_mean = np.array(f['te_mn'])
    argo_sa_mean = np.array(f['sa_mn'])
    te_model = np.array(f['te_modf'])
    sa_model = np.array(f['sa_modf'])
    pmodel = np.array(f['pmodel'])

['#refs#', 'ar', 'ba', 'dd', 'dpr', 'dyr', 'i', 'i1', 'i2', 'i3', 'ii', 'lat', 'lon', 'ltise', 'm', 'model', 'n', 'o', 'p', 'pmodel', 'pp', 'pr', 'q', 'r', 'sa_ano', 'sa_ju', 'sa_ju2', 'sa_mn', 'sa_mod', 'sa_modf', 'sa_res', 'sa_resf', 'sa_resff', 'te_ano', 'te_ju', 'te_ju2', 'te_mn', 'te_mod', 'te_modf', 'te_res', 'te_resf', 'te_resff', 'ti', 'vo', 'x', 'xx', 'y', 'yy']


In [26]:
%%time

# Reconstruct the total temperature and salinity fields using 
# the smoothed model coefficients and smoothed anomaly fields
te_tot = argo_temperature_anomaly_filt + argo_te_mean + te_model[0,:,:,:] + te_model[2,:,:,:]*np.reshape(np.sin(2*np.pi*dyr),(198,1,1,1)) + te_model[3,:,:,:]*np.reshape(np.cos(2*np.pi*dyr),(198,1,1,1)) + te_model[4,:,:,:]*np.reshape(np.sin(4*np.pi*dyr),(198,1,1,1)) + te_model[5,:,:,:]*np.reshape(np.cos(4*np.pi*dyr),(198,1,1,1))
sa_tot = argo_salinity_anomaly_filt + argo_sa_mean + sa_model[0,:,:,:] + sa_model[2,:,:,:]*np.reshape(np.sin(2*np.pi*dyr),(198,1,1,1)) + sa_model[3,:,:,:]*np.reshape(np.cos(2*np.pi*dyr),(198,1,1,1)) + sa_model[4,:,:,:]*np.reshape(np.sin(4*np.pi*dyr),(198,1,1,1)) + sa_model[5,:,:,:]*np.reshape(np.cos(4*np.pi*dyr),(198,1,1,1))


CPU times: user 13.7 s, sys: 10.9 s, total: 24.6 s
Wall time: 24.6 s


In [27]:
RG_filt = xr.Dataset({'te_anom_smooth': (('dyr', 'pressure', 'lat', 'lon'), argo_temperature_anomaly_filt),
                     'sa_anom_smooth': (('dyr', 'pressure', 'lat', 'lon'), argo_salinity_anomaly_filt),
                     'argo_te_mean': (('pressure', 'lat', 'lon'), argo_te_mean),
                     'argo_sa_mean': (('pressure', 'lat', 'lon'), argo_sa_mean),
                     'mask': (('pressure','lat', 'lon'), bathymetry_mask),
                     'te_modelf': (('coeff','pressure','lat','lon'), te_model),
                     'sa_modelf': (('coeff','pressure','lat','lon'), sa_model),
                     'te_tot': (('dyr', 'pressure', 'lat', 'lon'), te_tot),
                     'sa_tot': (('dyr', 'pressure', 'lat', 'lon'), sa_tot)},
                coords={'dyr': dyr, 'pressure': pressure ,'lat': lat, 'lon': lon, 'coeff': np.arange(1,7)})


In [29]:
# Save xarray dataset (RG_filt) to zarr
RG_filt.to_zarr("/glade/scratch/scanh/grl_2020_data/te_se_smooth_RG09_Johnson_170720.zarr")
